In [10]:
##### Python Scipy - Otimização
##### Notebook problema de otimização 
##### SAEE 2020 - Aprendizado de Máquna com Python - Oscar Schmitt Kremer.
##### A execucação de cada uma destas células é feita apertando o botão de play aqui à esquerda
##### Artigo: Differential Evolution – A Simple and Efficient Heuristic for global Optimization over Continuous Spaces
##### Autores: Rainer Storn & Kenneth Price 
##### Periódico: Journal of Global Optimization
import numpy as np
from scipy.optimize import differential_evolution

In [11]:
class SpringMass:
    '''
    Classe sobre sistemas 
    massa mola
    '''
    def __init__(self, K, B, M):
        self.K = K
        self.B = B
        self.M = M

In [16]:
def state_equations(t, x, sys, gains):
    '''
    Implementa a equação na 
    forma de variaveis de estado
    do sistema físico considerado.
    t: instante em um tempo específico
    x: vetor de estados do sistema
    sys: sistema massa mola
    gains: ganhos proporcional e derivativo
    '''
    x1 = x[0]
    x2 = x[1]
    xd = np.sin(t)
    xdponto = np.cos(t)
    u = gains['proporcional']*(xd-x1) + gains['derivative']*(xdponto-x2)
    if u > 100:
        u = 100       
    dx1dt=x2; 
    dx2dt= (1/sys.M)*(u - sys.B*x2 - sys.K*x1)
    return np.array([dx1dt,dx2dt])

In [13]:
def euler_solver(x, times, delta_t, sys, gains):
    '''
    Função que resolve a equação
    diferencial por Euler, fazendo a
    acumulação dos estados calculados com
    state_equation.
    x: estados iniciais
    times: vetor de tempo
    delta_t: discretização do tempo
    sys:sistema físico considerado
    gains: ganhos proporcional e derivativo
    '''
    x_plot = []
    for time in times:
        x = x + delta_t*state_equations(time, x, sys, gains)
        x_plot.append(x)
    return np.array(x_plot)

In [14]:
def itae(gains):
    '''
    Função que calcula o 
    ITAE(integral time absolute error)
    para um determinado caso de tempo,
    valor de referência e sistema 
    massa mola.
    gains
    '''
    delta_t = 1/100
    t = np.arange(0, 10, delta_t)
    x0 = np.array([0, 0])
    xd = np.sin(t)
    control_gains = {'proporcional': gains[0], 'derivative': gains[1]}
    sys = SpringMass(100, 8, 10)
    x1 = euler_solver(x0, t, delta_t, sys, control_gains)[:,0]
    if np.isnan(x1[-1]):
        return 10**6
    else:
        return sum(t*abs(xd-x1))

In [17]:
bounds = [(0.0001, 10000), (0.001, 10000)]
result = differential_evolution(itae, bounds, maxiter=100, popsize=30, disp=True)
print('Ganhos do algoritmo genetico'.format(result.x))
print('ITAE otimizado {}:'.format(result.fun))
print(itae([1000, 10]))
print(itae([1000, 200]))
print(itae([200, 1000]))
print(itae([10, 1000]))

differential_evolution step 1: f(x)= 42.7225
differential_evolution step 2: f(x)= 42.7225
differential_evolution step 3: f(x)= 42.7225
differential_evolution step 4: f(x)= 42.6435
differential_evolution step 5: f(x)= 42.5356
differential_evolution step 6: f(x)= 42.5356
differential_evolution step 7: f(x)= 42.5356
differential_evolution step 8: f(x)= 42.4894
differential_evolution step 9: f(x)= 42.4894
differential_evolution step 10: f(x)= 42.4859
differential_evolution step 11: f(x)= 42.4859
differential_evolution step 12: f(x)= 42.4859
Ganhos do algoritmo genetico
ITAE otimizado 42.45406792045475:
252.61162726615825
247.99364438215355
344.5025359013031
453.2935986777974
